<font color='blue'> 
### <u>КОММЕНТАРИЙ РЕВЬЮЕРА</u>
</font>

<font color='blue'>
<br />
Валерия, рад приветствовать тебя! <br />
<br />    

<br />
Меня зовут Николай. <br />И на этом этапе твоего движения к новой профессии от меня будут комментарии по написанному коду. <br />Чтобы меньше "мусорить эфир" и чтобы максимально наглядно отделяться от основного кода, есть предложение условиться в некоторых обозначениях:<br /> <br />
<u><b>ТАКОЙ ШРИФТ</b></u> - всегда начало комментария <br />
<font color='green'>такой шрифт</font> - комментарии о том, что всё ОК <br/>
<font color='orange'>такой шрифт</font> - комментарии о том, что всё ОК по результату,<br> однако есть на что обратить внимание в плане применения техник, или есть способы сделать более короткий или быстрый код <br/>
<font color='red'>такой шрифт</font> - комментарии о том, что есть критичный момент, влияющий на бизнес-результат проекта.<br/>
</font><br /><br />

# <a id='7'>Отток клиентов</a>

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# Оглавление
- [План работы](#1)
- [Подготовка данных](#2)
- [Исследование задачи](#3)
- [Борьба с дисбалансом](#4)
- [Тестирование модели](#5)
- [Итоговый вывод](#6)

## <a id='1'>План работы</a>

- [Перейти в начало проекта](#7)
- [Перейти к следующему блоку](#2)

Прогнозирование ухода клиента из банка - задача классификации, где у нас есть два варианта: 0 - клиент не уйдет, 1 - клиент уйдет. <br>
План работы над проектом:
1. Подготовить данные к работе - удалить лишние признаки, обработать пропуски, закодировать категориальные признаки. 
2. Исследуем баланс классов и обучим модели без учета дисбаланса. Проверим результаты, сделаем выводы. 
3. Улучшим качество модели, учитывая дисбаланс классов. Найдем и обучим лучшую модель. Для оценки модели будем использовать F1-меру и AUC-ROC. 
4. Проведем финальное тестирование модели на тестовой выборке. 

# <a id='2'>1. Подготовка данных</a>

- [Перейти в начало проекта](#7)
- [Перейти к предыдущему блоку](#1)
- [Перейти к следующему блоку](#3)

## Обзор данных

Импортируем все необходимые для работы библиотеки. 

In [12]:
import pandas as pd 
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix

Импортируем датасет, посмотрим на первые строки и на общую информацию о датасете. 

In [13]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


**У нас есть признаки**: 
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — количество недвижимости у клиента
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата<br>

**И целевой признак**:
- Exited — факт ухода клиента

**Первичные наблюдения**: 
- Мы видим пропущенные значения в столбце `Tenure`, которые необходимо заполнить. 
- У нас есть столбец `Surname`, от которого не будет зависеть уход пользователя из банка. Его мы можем удалить, чтобы не плодить лишние признаки. 
- То же самое относится к уникальному идентификатору клиента `CustomerId`, который не может влиять на решение клиента об уходе из банка. 
- Также есть бесполезный для обучения модели столбец `RowNumber` с индексом строки в данных. В нашем датафрейме уже есть индексы, которые нас полностью устраивают. Этот столбец надо удалить. 
- У нас есть два категориальных столбца `Geography` и `Gender`, которые нужно закодировать при помощи OHE-кодирования для корректной работы моделей. 
- Также нам необходимо масштабировать количественные признаки с помощью стандартизации данных, чтобы избежать неправильного распределения важности признаков. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
хороший план)

## Обработка данных

### Удаление столбцов

У нас есть три признака, которые не должны влиять на предсказания модели. Это: `Surname`, `CustomerId`, `RowNumber`. Их мы удалим. 

In [15]:
data = data.drop(['Surname', 'CustomerId', 'RowNumber'], axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


Удаление столбцов прошло успешно. Модель не будет путаться, присваивая 'Ивановым' повышенный риск ухода из банка. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
так их, эти столбцы)

### Обработка пропусков 

Мы обнаружили пропущенные значения в столбце `Tenure`. Это столбец с количеством недвижимости у клиента. Мы можем допустить несколько вариантов: 
- Отсутствие значения = отсутствие недвижимости. 
- Отсутствие значения - результат какого-либо сбоя или ошибки. <br>

Достоверной информации у нас нет. Поэтому у нас есть два варианта заполнения пропусков:
1. Заполнение пропусков **нестандартными значениями**. Например, -1. Но нам известно, что модель логистической регрессии плохо работает с нестандартными значениями. 
2. Заполнение пропусков **средними/медианными значениями**. С этими значениями хорошо работают все модели. Выберем этот способ. 

In [16]:
data['Tenure'].describe()

count    9091.000000
mean        4.997690
std         2.894723
min         0.000000
25%         2.000000
50%         5.000000
75%         7.000000
max        10.000000
Name: Tenure, dtype: float64

Среднее и медианные значения очень близки. Заполним пропуски медианным значением. 

In [17]:
data['Tenure'] = data['Tenure'].fillna(data['Tenure'].median())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             10000 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(6), object(2)
memory usage: 859.5+ KB


In [18]:
data['Tenure'].describe()

count    10000.00000
mean         4.99790
std          2.76001
min          0.00000
25%          3.00000
50%          5.00000
75%          7.00000
max         10.00000
Name: Tenure, dtype: float64

Все пропуски заполнены, структура данных не пострадала. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
Достаточно обоснованное решение) <br>
<br> Возможно, что групповая (например по географии) медиана также не принесла бы большой ошибки. А может взять и удалить, чтобы не "искушаться"... Всем же понятно, что однозначного решения здесь нет, как и алгоритма. Правило только одно (как в медицине) - НЕ нарведи. Т.е. принимаемое решение должно соотноситься с возможным ущербом на конечный результат. В нашем случае ущерб от медианного заполнения ниже от ущерба удалить данные?. Причём выше/ниже - это только на чуйке/жизенном опыте смого аналитика.<br>
Интересная это работа - быть DS-аналитиком)<br>
    
    
В данном случае, спарведливости ради, если построить распределение ДО и ПОСЛЕ заполнения пропусков, то увидим, что оно изменится. Т.е. данный подход несколько исказит инфо.<br> Тут лучшим способом будет - заполнение случайным числом.<br>
    
Ещё вараинт:<br> удалить пропущенные строки. А по окнчании построения модели, вывести важность факторов самой модели. И если фактор tenure окажется не значим, то как в принципе удалить его из моделирования, а первоначально удалённые строки - вернуть.

### OHE-кодирование

У нас есть два категориальных столбца `Geography` и `Gender`, которые нужно закодировать при помощи OHE-кодирования для корректной работы моделей.

In [19]:
data = pd.get_dummies(data, drop_first=True)
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2.0,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1.0,83807.86,1,0,1,112542.58,0,0,1,0
2,502,42,8.0,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1.0,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2.0,125510.82,1,1,1,79084.10,0,0,1,0


OHE-кодирование признаков прошло успешно. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
ОК)

### Масштабирование признаков

Масштабируем количественные признаки. 

In [20]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(data[numeric])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [21]:
data[numeric] = scaler.transform(data[numeric])
data.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain,Gender_Male
0,-0.326221,0.293517,-1.086246,-1.225848,-0.911583,1,1,0.021886,1,0,0,0
1,-0.440036,0.198164,-1.448581,0.117350,-0.911583,0,1,0.216534,0,0,1,0
2,-1.536794,0.293517,1.087768,1.333053,2.527057,1,0,0.240687,1,0,0,0
3,0.501521,0.007457,-1.448581,-1.225848,0.807737,0,0,-0.108918,0,0,0,0
4,2.063884,0.388871,-1.086246,0.785728,-0.911583,1,1,-0.365276,0,0,1,0


<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
хорошо)

### Деление данных на выборки 

Разобьем данные на признаки и целевой признак, а также разделим на тренировочную и тестовую выборки. Деление на тренировочную и валидационную выборки будем производить в дальнейшем при обучении моделей (с помощью GridSearchCV). Под тестовую выборку выделим 10% данных. Чем больше данных мы оставим для обучения, тем лучше обучится модель. 

In [22]:
features = data.drop(['Exited'], axis=1)
target = data['Exited']

features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.1, random_state=12)
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

(9000, 11)
(1000, 11)
(9000,)
(1000,)


Датасет успешно разделен на выборки. Можно приступать к работе. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
отлично и проконтролировали размер выборок)

# Вывод

Мы подготовили данные к работе с моделями машинного обучения:
1. Удалили лишние признаки
2. Заполнили пропуски в столбце `Tenure`-  пропусков было почти 10%. Нужно проверить, по какой причине это произошло (сбой системы, члеовеческий фактор и т.д.)
3. Провели OHE-кодирование для категориальных признаков.
4. Провели стандартизацию признаков.
4. Разделили данные на тренировочную и тестовую выборки. 

Данные готовы к работе с моделями машинного обучения. Можно приступать к работе. 

# <a id='3'>2. Исследование задачи</a>

- [Перейти в начало проекта](#7)
- [Перейти к предыдущему блоку](#2)
- [Перейти к следующему блоку](#4)

Исследуем баланс классов. Посмотрим, как много значений целевого признака каждого класса в нашей обучающей выборке. 

In [23]:
target_train.value_counts(normalize=True)

0    0.797333
1    0.202667
Name: Exited, dtype: float64

80% нашей выборки составляет значения отрицательного класса. Вероятно, наша модель будет отлично предсказывать отрицательные значения и плохо предсказывать положительные значения. То есть будет плохо выполнять задачу предсказания оттока клиентов (значение 1 = уход клиента из банка). Проверим это на практике чуть позже. Предварительно обучим модели и выберем лучшую. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
принято)

#### Дерево решений

Обучим модель Дерева решений, выберем лучший параметр глубины с помощью GridSearchCV.

In [24]:
param_grid = {'max_depth': [depth for depth in range(1, 16, 1)]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=12), param_grid=param_grid)

In [25]:
%%time
gs.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 1.31 s, sys: 1.09 ms, total: 1.31 s
Wall time: 1.41 s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=12,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15]},
  

In [15]:
gs.best_params_

{'max_depth': 6}

Лучший параметр глубины - 6. Проведем оценку модели с этими параметрами. 

In [16]:
dt_model = DecisionTreeClassifier(random_state=12, max_depth=6)
dt_model.fit(features_train, target_train)
predicted_test = dt_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = dt_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.97      0.87      0.92       878
           1       0.48      0.84      0.61       122

    accuracy                           0.87      1000
   macro avg       0.73      0.85      0.77      1000
weighted avg       0.91      0.87      0.88      1000

Значение F1 (без разбивки по классам): 0.608955223880597
Значение AUC-ROC: 0.8405396376565194


Мы получили достаточно хорошее значение F1 - выше минимально допустимого (0.59). Проверим другие модели.  

#### Случайный лес

Обучим модель случайного леса, выберем лучшие параметры модели с помощью GridSearchCV. 

In [17]:
param_grid = {'max_depth': [depth for depth in range(1, 16, 1)],
             'n_estimators': [estim for estim in range(20, 101, 10)]}
gs = GridSearchCV(RandomForestClassifier(random_state=12), param_grid=param_grid)

In [18]:
%%time
gs.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 2min 43s, sys: 608 ms, total: 2min 44s
Wall time: 2min 46s


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False, random_state=12,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_gr

In [19]:
gs.best_params_

{'max_depth': 9, 'n_estimators': 60}

Протестируем модель с заданными параметрами. 

In [20]:
rf_model = RandomForestClassifier(random_state=12, max_depth=9, n_estimators=60)
rf_model.fit(features_train, target_train)
predicted_test = rf_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = rf_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.98      0.87      0.92       885
           1       0.47      0.88      0.62       115

    accuracy                           0.87      1000
   macro avg       0.73      0.88      0.77      1000
weighted avg       0.92      0.87      0.89      1000

Значение F1 (без разбивки по классам): 0.6158536585365855
Значение AUC-ROC: 0.8856595737065341


Значение метрики F1 незначительно, но все-таки выше, чем у модели Дерева решений(0.61 против 0.60). Значение AUC-ROC тоже выше (0.88 против 0.84). Проверим показатели модели логистической регрессии.  

#### Логистическая регрессия

In [21]:
lr_model = LogisticRegression(random_state=12)
lr_model.fit(features_train, target_train)
predicted_test = lr_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = lr_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.98      0.83      0.90       934
           1       0.23      0.76      0.36        66

    accuracy                           0.82      1000
   macro avg       0.61      0.79      0.63      1000
weighted avg       0.93      0.82      0.86      1000

Значение F1 (без разбивки по классам): 0.35842293906810035
Значение AUC-ROC: 0.7833694245097864


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


У модели логистической регрессии, вполне ожидаемо, наихудшие показатели. Модель логистической регрессии лучше работает с данными, в которых значительный перевес количественных признаков, которые проходят сравнение на больше/меньше. В наших данных очень много категориальных признаков. 

#### Выбор модели

Из всех моделей лучшие показатели у модели Случайного леса. Достаточно высокое значение AUC-ROC (0.88) и достаточное для прохождения минимального порога значение F1 - 0.61.

# Вывод

При тестировании моделей лучшие результаты показала модель случайного леса с параметрами **max_depth=9** и **n_estimators=60**. <br>
AUC-ROC модели = 0.88, **F1 (ключевой показатель) = 0.61** (выше порогового 0.59).<br>
Попробуем улучшить показатели модели с помощью балансировки классов. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
хорошая работа с моделями)

# <a id='4'>3. Борьба с дисбалансом</a>

- [Перейти в начало проекта](#7)
- [Перейти к предыдущему блоку](#3)
- [Перейти к следующему блоку](#5)

Для борьбы с дисбалансом классов мы будем использовать два метода:
1. Пропишем class_weight в параметры модели
2. Увеличим выборку для баланса классов

#### Class_weight

Пропишем параметр class_weight='balanced' для модели, выбранной в предыдущем блоке.

In [22]:
rf_model = RandomForestClassifier(random_state=12, max_depth=9, n_estimators=60, class_weight='balanced')
rf_model.fit(features_train, target_train)
predicted_test = rf_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = rf_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       762
           1       0.69      0.62      0.66       238

    accuracy                           0.84      1000
   macro avg       0.79      0.77      0.78      1000
weighted avg       0.84      0.84      0.84      1000

Значение F1 (без разбивки по классам): 0.656319290465632
Значение AUC-ROC: 0.8718196514964416


Ключевой показатель оценки нашей модели действительно вырос - с 0.61 до 0.65. Проверим второй способ. 

#### Увеличение выборки

Разделим выборки на строки с положительным классом целевого признака и отрицательным. 

In [23]:
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

Увеличим количество строк с положительным классом целевого признака. 

In [24]:
repeat = 4
features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

Перемешаем строки в выборке, чтобы положительные и отрицательные строки не шли друг за другом, - это плохо влияет на обучение модели. 

In [25]:
features_upsampled, target_upsampled = shuffle(features_upsampled, 
                                                   target_upsampled, 
                                                   random_state=12)

In [26]:
target_upsampled.value_counts(normalize=True)

1    0.504146
0    0.495854
Name: Exited, dtype: float64

Обучим модель из предыдущего блока и сравним показатели. 

In [27]:
rf_model = RandomForestClassifier(random_state=12, max_depth=9, n_estimators=60)
rf_model.fit(features_upsampled, target_upsampled)
predicted_test = rf_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = rf_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       715
           1       0.75      0.56      0.64       285

    accuracy                           0.82      1000
   macro avg       0.79      0.74      0.76      1000
weighted avg       0.81      0.82      0.81      1000

Значение F1 (без разбивки по классам): 0.6385542168674698
Значение AUC-ROC: 0.876615900400284


Значение ключевого показателя F1 увеличилось с 0.61 до 0.63, при этом оно не выше, чем при балансировке классов с помощью параметра class_weight. 

#  Вывод

Для балансировки классов параметр class_weight со значением 'balanced' показал результат лучше, чем увеличение выборки. В итоговой модели будем использовать его. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
есть, здорово)

# <a id='5'>4. Тестирование модели</a>

- [Перейти в начало проекта](#7)
- [Перейти к предыдущему блоку](#4)
- [Перейти к итоговому выводу](#6)

Проведем итоговое тестирование нашей модели на тестовой выборке. <br>
Мы выбрали модель Случайного леса с параметрами max_depth=9 и n_estimators=60, а также использовали class_weight='balanced' для балансировки классов. 

In [28]:
rf_model = RandomForestClassifier(random_state=12, max_depth=9, n_estimators=60, class_weight='balanced')
rf_model.fit(features_train, target_train)
predicted_test = rf_model.predict(features_test)
print(classification_report(predicted_test, target_test))
print('Значение F1 (без разбивки по классам):', f1_score(target_test, predicted_test))
probabilities_valid = rf_model.predict_proba(features_test)
probabilities_one_valid = probabilities_valid[:, 1]
print('Значение AUC-ROC:', roc_auc_score(target_test, probabilities_one_valid))

              precision    recall  f1-score   support

           0       0.89      0.91      0.90       762
           1       0.69      0.62      0.66       238

    accuracy                           0.84      1000
   macro avg       0.79      0.77      0.78      1000
weighted avg       0.84      0.84      0.84      1000

Значение F1 (без разбивки по классам): 0.656319290465632
Значение AUC-ROC: 0.8718196514964416


Наша модель соответствует заявленным требованиям - показатель F1 более 0.59. Модель прошла итоговое тестирование. 

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
ОК, хороший забег по моделям)) Ты использовала кросс-валидацию, поэтому, теоретически действиетльно три выборки и не нужны получается.<br>
Так и выходит, что grid-ом подбирала модель, а потом тестировала на test. А потом в пункте 4 опять тестировала на test)))<br>
Масло масленное:) т.е. 4 пункт и не нужен в данной логике.<br>
    
Просто задание замышлялось, что студент не будет использовать автоподбор параметров, а организует ручной (в цикле) перебор параметров. Если твк двигаться, то пункт 4 будет нужен.<br>

# <a id='6'>Итоговый вывод</a>

Мы подготовили данные к работе с моделями машинного обучения:
- Удалили лишние признаки
- Заполнили пропуски в столбце Tenure- пропусков было почти 10%. Нужно проверить, по какой причине это произошло (сбой системы, члеовеческий фактор и т.д.)
- Провели OHE-кодирование для категориальных признаков.
- Провели стандартизацию признаков.

Мы выявили дисбаланс классов и использовали параметр class_weight='balanced' для балансировки.
Лучшие результаты при оценке показала модель Случайного леса с параметрами max_depth=9 и n_estimators=60. Она успешно прошла тестирование на тестовой выборке и показала результат оценки F1 = 0.65, что выше порогового значения, равного 0.59. Также модель показала самое высокое значение метрики AUC-ROC, равное 0.87, что является достаточно хорошим показателем для модели (в сравнении со случайной моделью, значение метрики AUC-ROC которой равно 0.5).

<font color='blue'><u><b>КОММЕНТАРИЙ РЕВЬЮЕРА</b></u></font><br />
<font color='green'>
<br/> Валерия, добротная, качественная работа! <br>
Подготовка данных, работа с диссбалансом и исследование моделей и поиск парамтров для них - ничего не забыто <br>
И видно, что проделана рабочая аналитическая работа: результативно и с выводами.<br>  

<font color='green'>
Если говорить, что можно ещё улучшить с проекте, ну и использовть в подобных проектах, то я бы выделили такие моменты:<br>
1) После подбора параметров модели, когда уже настаёт этап тестировани. Можно обучить на обобщённой выборке (трайн + валидация) ну и тестировать потом. Это может ещё прибавить точности. (это если не использовать grid)<br>
    
2) Для понимания, а какие в итоге факторы важны при моделировании, можно выводить их важность, используя feature_importances_, ну и график заодно. Это будет смотреться очнь здорово: результативная метрика и график важности факторов)))<br>
    
3) Наверное ты в курсе, просто хочу напомнить: F1 и AUC - отличные метрики, но нужно держать в голове, что это лишь бобщённые характеристики. А в реальной практике бизнесу часто важно знать детали самой ошибки - из каких ошибок/успехов она складывается. И в этом случае очень кстати будут меры полноты и точности. Рассмотри возможность их использования для своих проектов. Только через них можно выйти на финансовые возможности модели. Ведь перед запуском её в бой, будет сравнение стоимости создания и поддержки модели со стомостью "оставить как есть" <br>
    
4) предварительная работа с данными:<br>
- На этом этапе нужно БОЛЬШЕ аргументации, для принимаемых решений.<br>
- Также, можно организовать для себя некую систему работы с данными:<br>
 
Например, алгоритм такой может:<br>
1) анализ аномалий <br>
2) анализ пропусков <br>
3) проверка дубликатов<br>
4) Анализ факторов: исходя из типа шкалы, могут применять следующие графический и аналитический анализы:<br>
- колличественная-колличественная: аналитически методод - корреляции, графический - скатерплот.
- колличественная-номинальная: аналитически методод - проверка гипотез (стьюдент, маннауитни), графический - боксплот.
- номинальная-номинальная: аналитически методод - Хи-квадрат, графический - круговая диаграмма
<br><br>
УСПЕШНОГО ДАЛЬНЕЙШЕГО ОБУЧЕНИЯ!

</b></font>

### [Перейти в начало проекта](#7)

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнен шаг 1: данные подготовлены
- [ ]  Выполнен шаг 2: задача исследована
    - [ ]  Исследован баланс классов
    - [ ]  Изучены модели без учёта дисбаланса
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 3: учтён дисбаланс
    - [ ]  Применено несколько способов борьбы с дисбалансом
    - [ ]  Написаны выводы по результатам исследования
- [ ]  Выполнен шаг 4: проведено тестирование
- [ ]  Удалось достичь *F1*-меры не менее 0.59
- [ ]  Исследована метрика *AUC-ROC*